# FString Chains

In [ ]:
import os

if os.environ.get("TOGETHER_API_KEY") is None:
   os.environ["TOGETHER_API_KEY"] = "paste_your_api_key_here"

In [ ]:
!pip3 install langchain langchain_openai

In [ ]:
import os
from langchain_openai import ChatOpenAI

model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT"
#model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
#model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-Instruct-v0.2"
#model_name = "openchat/openchat-3.5-1210"
#model_name = "Qwen/Qwen1.5-72B-Chat"

print("Provider: TogetherAI")
print("Model: " + model_name)

llm=ChatOpenAI(model_name=model_name,
               openai_api_key=os.environ.get("TOGETHER_API_KEY"),
               openai_api_base="https://api.together.xyz/v1/")

In [ ]:
import textwrap
from langchain_core.prompts import ChatPromptTemplate

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    text = text.lstrip()
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []
    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines
    return '\n'.join(wrapped_lines)

def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(wrap_text(prompt))
    print("")
    print("Response: ")
    print(response)

def print_messages_and_response(messages, response):
    prompt = ChatPromptTemplate(messages=messages)
    print_prompt_and_response(prompt.format(), response)

def get_completion(prompt, temperature=0.0):
    response = llm.invoke(prompt, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

def get_completion_messages(messages, temperature=0.0):
    response=llm.invoke(messages, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

Here is an example of a simple chain with 2 calls to an LLM. The first
call asks for 3 names of a company, then the second call asks the LLM to
take the response from the first call and upper case it.

This represents a chain of calls to LLMs and we want to show how to do
this with F-strings.

Its easy if we dont care about re-using the the chain. Here is how to
achieve it with a couple of FStrings.

Run the version below which is created with 2 F-strings. Each one is the
prompt that will be passed to the LLM for that stage of the chain.

In [ ]:
thing = "toothpicks"

first_query = f"""
Give me 3 possible names for companies that make {thing}.
Just list the 3 names like this:
1 <Company 1 name>
2 <Company 2 name>
3 <Company 3 name>
"""

first_response = get_completion(first_query)
print(first_response)

second_query = f"""
Can you upper case the text below:
Just uppercase the text and return the uppercased version.

{first_response}

"""
second_response = get_completion(second_query)
print(second_response)

In [ ]:
def chain(thing):
  first_query = f"""
  Give me 3 possible names for companies that make {thing}.
  Just list the 3 names like this:
  1 <Company 1 name>
  2 <Company 2 name>
  3 <Company 3 name>
  """

  first_response = get_completion(first_query)
  print(first_response)
  print(80*'-')

  second_query = f"""
  Can you upper case the text below:
  Just uppercase the text and return the uppercased version.

  {first_response}

  """

  second_response = get_completion(second_query)
  return second_response

response=chain("bikes")
print(response)

print(80*"*")

response=chain("firecrackers")
print(response)

print(80*"*")

response=chain("yoga mats")
print(response)